In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import re

In [6]:
pd_data = pd.read_csv('미세먼지_PM10__월별_도시별_대기오염도_20230322213853.csv', encoding='cp949', header=[1])

# '시점' '서울특별시' '부산광역시' '대전광역시' '제주특별자치도'

pd_data = pd_data.astype(str)

date = pd_data['시점']
seoul_data = pd_data['서울특별시'].str.replace(r'[*]+', repl= r'', regex=True)
busan_data = pd_data['부산광역시'].str.replace(r'[*]+', repl= r'', regex=True)
daejeon_data = pd_data['대전광역시'].str.replace(r'[*]+', repl= r'', regex=True)
jeju_data = pd_data['제주특별자치도'].str.replace(r'[*]+', repl= r'', regex=True)

pd_data = pd.concat((date, seoul_data,busan_data,daejeon_data,jeju_data), axis=1)
# print(pd_data)


np_data = pd_data.to_numpy()


np_data = np_data.astype(np.float64)

# print(np.where(np.isin(np_data, r'[^0-9]+')))
# print(np.where(np.isin(np_data, '*')))

print(np_data)

[[2011.01   44.     38.     39.     29.  ]
 [2011.02   75.     57.     63.     52.  ]
 [2011.03   65.     58.     59.     48.  ]
 [2011.04   56.     56.     52.     50.  ]
 [2011.05   72.     76.     67.     76.  ]
 [2011.06   44.     52.     43.     43.  ]
 [2011.07   28.     41.     28.     28.  ]
 [2011.08   27.     35.     25.     29.  ]
 [2011.09   29.     33.     28.     26.  ]
 [2011.1    42.     43.     42.     40.  ]
 [2011.11   38.     41.     40.     36.  ]
 [2011.12   46.     41.     43.     36.  ]
 [2012.01   60.     45.     51.     47.  ]
 [2012.02   50.     46.     45.     39.  ]
 [2012.03   46.     47.     46.     45.  ]
 [2012.04   51.     53.     49.     41.  ]
 [2012.05   52.     58.     53.     51.  ]
 [2012.06   40.     38.     37.     23.  ]
 [2012.07   28.     40.     27.     21.  ]
 [2012.08   22.     34.     19.     19.  ]
 [2012.09   27.     35.     28.     31.  ]
 [2012.1    33.     42.     33.     37.  ]
 [2012.11   42.     47.     44.     44.  ]
 [2012.12  

#### 시도별 평균 미세먼지 농도 차이 분석 --> ANOVA 사용(지역이 4개소이므로!)

In [7]:
# 평균값 구하기

seoul_dust = np.mean(np_data[:,1])
busan_dust = np.mean(np_data[:,2])
daejeon_dust = np.mean(np_data[:,3])
jeju_dust = np.mean(np_data[:,4])

print(f"서울 : {seoul_dust}\n부산 : {busan_dust}\n대전 : {daejeon_dust}\n제주 : {jeju_dust}\n")

서울 : 42.79545454545455
부산 : 41.84848484848485
대전 : 41.20454545454545
제주 : 39.70454545454545



In [8]:
# ANOVA
list_d = [np_data[:,x] for x in range(1,5)]

print(stats.f_oneway(*list_d))

# p-value > 0.05를 만족하므로 귀무가설 수용

F_onewayResult(statistic=1.4500073764107557, pvalue=0.2273752066976529)
[array([44., 75., 65., 56., 72., 44., 28., 27., 29., 42., 38., 46., 60.,
       50., 46., 51., 52., 40., 28., 22., 27., 33., 42., 41., 64., 45.,
       55., 52., 55., 40., 34., 35., 28., 29., 42., 55., 57., 57., 60.,
       58., 63., 38., 38., 29., 29., 33., 45., 44., 49., 84., 71., 45.,
       45., 35., 30., 34., 28., 44., 33., 48., 50., 45., 64., 71., 56.,
       46., 33., 34., 37., 38., 52., 48., 53., 46., 60., 56., 63., 41.,
       33., 21., 32., 29., 42., 50., 52., 53., 52., 52., 42., 37., 24.,
       22., 20., 28., 52., 43., 66., 57., 69., 41., 52., 29., 26., 25.,
       21., 31., 40., 42., 42., 41., 45., 44., 35., 36., 21., 23., 19.,
       33., 42., 42., 38., 48., 67., 42., 61., 33., 24., 22., 15., 27.,
       45., 39.]), array([38., 57., 58., 56., 76., 52., 41., 35., 33., 43., 41., 41., 45.,
       46., 47., 53., 58., 38., 40., 34., 35., 42., 47., 36., 56., 47.,
       66., 49., 62., 47., 47., 47., 33., 38

#### 2011~2021년 봄(3~5월), 겨울(10~12월)의 평균 미세먼지 농도 차이 분석 --> TTEST 사용(그룹이 2개이므로!)

In [9]:
list_y = [float(x) for x in range(2011,2022)]
list_s = [0.03, 0.04, 0.05]
list_w = [0.1, 0.11, 0.12]


np_spring_data = np.array([])
for i in list_y :
  for k in list_s :
    filter_s = np.isin(np_data[:, 0], i+k)
    np_spring_data = np.append(np_spring_data, np_data[:,1:][filter_s])
    # print(np_data[:,1:][filter_s])

np_winter_data = np.array([])
for i in list_y :
  for k in list_w :
    filter_s = np.isin(np_data[:, 0], i+k)
    np_winter_data = np.append(np_winter_data, np_data[:,1:][filter_s])
    # print(np_data[:,1:][filter_s])

print(np_spring_data)
print(np_winter_data)


[65. 58. 59. 48. 56. 56. 52. 50. 72. 76. 67. 76. 46. 47. 46. 45. 51. 53.
 49. 41. 52. 58. 53. 51. 55. 66. 54. 56. 52. 49. 48. 46. 55. 62. 53. 58.
 60. 50. 50. 57. 58. 53. 45. 51. 63. 71. 59. 81. 71. 56. 68. 62. 45. 48.
 40. 48. 45. 56. 49. 42. 64. 52. 59. 53. 71. 60. 64. 61. 56. 53. 49. 56.
 60. 52. 54. 47. 56. 54. 58. 59. 63. 58. 63. 50. 52. 44. 52. 43. 52. 63.
 60. 56. 42. 44. 47. 38. 69. 51. 68. 56. 41. 40. 40. 43. 52. 47. 46. 47.
 45. 33. 40. 39. 44. 36. 41. 44. 35. 32. 32. 38. 67. 60. 63. 76. 42. 37.
 41. 37. 61. 36. 56. 44.]
[42. 43. 42. 40. 38. 41. 40. 36. 46. 41. 43. 36. 33. 42. 33. 37. 42. 47.
 44. 44. 41. 36. 38. 26. 29. 38. 30. 39. 42. 43. 39. 50. 55. 47. 46. 45.
 33. 39. 29. 38. 45. 47. 39. 40. 44. 42. 43. 49. 44. 43. 50. 51. 33. 33.
 37. 41. 48. 41. 49. 42. 38. 37. 36. 42. 52. 49. 51. 43. 48. 44. 47. 38.
 29. 32. 36. 30. 42. 50. 51. 45. 50. 46. 49. 37. 28. 32. 31. 31. 52. 47.
 63. 42. 43. 40. 51. 33. 31. 27. 29. 34. 40. 35. 43. 44. 42. 35. 40. 33.
 33. 30. 33. 34. 42. 31. 

In [10]:
def do_mean_comparison(g1, g2) :
  print("*******************************")
  print('g1 sharpiro : ', stats.shapiro(g1))
  print('g2 shapiro : ', stats.shapiro(g2))
  print('levene : ', stats.levene(g1, g2))
  print('ttest_ind : ', stats.ttest_ind(g1, g2))
  print("*******************************")

In [11]:
# 평균값 차이 비교 (By t-test)
do_mean_comparison(np_spring_data, np_winter_data)

*******************************
g1 sharpiro :  ShapiroResult(statistic=0.989180326461792, pvalue=0.39251869916915894)
g2 shapiro :  ShapiroResult(statistic=0.9878075122833252, pvalue=0.29399433732032776)
levene :  LeveneResult(statistic=11.01025439381632, pvalue=0.0010345437486875143)
ttest_ind :  Ttest_indResult(statistic=12.232958045607598, pvalue=1.6003353734189516e-27)
*******************************


In [12]:
def get_95ci(vals) :
  vals_mean = np.mean(vals)
  vals_std = np.std(vals)
  print('upper : ', vals_mean + 1.96*vals_std)
  print('lower : ', vals_mean - 1.96*vals_std)
  print(vals_mean - 1.96*vals_std, '~', vals_mean, '~', vals_mean + 1.96*vals_std)

In [13]:
get_95ci(np_spring_data)
print("**********************************************************")
get_95ci(np_winter_data)

upper :  72.33428120795399
lower :  33.01420364053085
33.01420364053085 ~ 52.67424242424242 ~ 72.33428120795399
**********************************************************
upper :  53.606421535552826
lower :  25.332972403841115
25.332972403841115 ~ 39.46969696969697 ~ 53.606421535552826


#### * 다른 풀이

#### 2021년 시도별 평균 미세먼지 농도 차이 분석 --> ANOVA 사용(지역이 4개소이므로!)

#### 2011~2021년 봄(3~5월), 겨울(10~12월)의 평균 미세먼지 농도 차이 분석 --> TTEST 사용(그룹이 2개이므로!)

In [14]:
fn = '미세먼지_PM10__월별_도시별_대기오염도_20230322213853.csv'

# '시점' '서울특별시' '부산광역시' '대전광역시' '제주특별자치도'

np_data = pd.read_csv(fn, encoding='cp949').to_numpy()
# print(np_data)

col_names = np_data[0]
# print(col_names)

row_index = np_data[1:,0]
# print(row_index)

np_data = np_data[1:, 1:]
# print(np_data, np_data.shape)
row_count, col_count = np_data.shape  # tuple
# print(row_count, col_count)

# flatten, ravel  : 2D array -> 1D array로 변환
# flatten : 복사본 생성
# ravel : 원본 참조

# 2차원 배열은 1차원 배열로 변환하여 전처리 후 reshape
np_data_flat = np_data.flatten()
# print(np_data_flat, np_data_flat.shape)

for i, value in enumerate(np_data_flat) :
  if '*' in value :
    star_index = value.find('*')
    np_data_flat[i] = value[:star_index]

np_data_flat_re = np_data_flat.reshape(132,4)

np_data = np_data_flat_re.astype(np.int64)

print(np_data)

[[44 38 39 29]
 [75 57 63 52]
 [65 58 59 48]
 [56 56 52 50]
 [72 76 67 76]
 [44 52 43 43]
 [28 41 28 28]
 [27 35 25 29]
 [29 33 28 26]
 [42 43 42 40]
 [38 41 40 36]
 [46 41 43 36]
 [60 45 51 47]
 [50 46 45 39]
 [46 47 46 45]
 [51 53 49 41]
 [52 58 53 51]
 [40 38 37 23]
 [28 40 27 21]
 [22 34 19 19]
 [27 35 28 31]
 [33 42 33 37]
 [42 47 44 44]
 [41 36 38 26]
 [64 56 58 44]
 [45 47 47 37]
 [55 66 54 56]
 [52 49 48 46]
 [55 62 53 58]
 [40 47 40 36]
 [34 47 28 30]
 [35 47 28 31]
 [28 33 29 34]
 [29 38 30 39]
 [42 43 39 50]
 [55 47 46 45]
 [57 57 56 56]
 [57 45 51 48]
 [60 50 50 57]
 [58 53 45 51]
 [63 71 59 81]
 [38 54 39 54]
 [38 48 31 41]
 [29 36 23 35]
 [29 39 27 34]
 [33 39 29 38]
 [45 47 39 40]
 [44 42 43 49]
 [49 47 52 51]
 [84 63 68 63]
 [71 56 68 62]
 [45 48 40 48]
 [45 56 49 42]
 [35 49 40 41]
 [30 37 31 36]
 [34 39 32 39]
 [28 29 27 33]
 [44 43 50 51]
 [33 33 37 41]
 [48 41 49 42]
 [50 45 48 45]
 [45 46 46 53]
 [64 52 59 53]
 [71 60 64 61]
 [56 53 49 56]
 [46 39 38 38]
 [33 29 27

In [15]:
# 2차원 배열 그대로 전처리하는 방법

# for row_index in range(row_count) :
#   for col_index in range(col_count) :
#     value_str = np_data[row_index, col_index]
#     if '*' in value_str :
#       # *가 위치한 인덱스를 찾아 인덱스 슬라이싱 활용(end point로 찾은 인덱스 적용)
#       star_index = value_str.find('*')
#       np_data[row_index, col_index] = value_str[:star_index]
#     # print(np_data[row_index, col_index])

In [16]:
# np_data_rav = np_data.ravel()
# # print(np_data_flat, np_data_flat.shape)

# for i, value in enumerate(np_data_rav) :
#   if '*' in value :
#     star_index = value.find('*')
#     np_data_rav[i] = value[:star_index]

# # print(np_data_flat_re)

# np_data = np_data_rav.astype(np.int64)

In [17]:
# test code
test_str_a = '123*'
test_str_b = '123**'

print('*' in test_str_a)
print('*' in test_str_b)

print(test_str_a.find('*'))

print(test_str_a[:3])

True
True
3
123


In [18]:
# print(col_names)
# print(row_index)
# print(np_data)

# 평균값 구하기
data_2021 = np_data[-12:,:]
# print(data_2021, data_2021.shape)

mean_2021 = np.mean(data_2021, axis=0)
print(mean_2021)

[38.41666667 31.16666667 35.5        33.16666667]


In [19]:
# ANOVA
anova_result = stats.f_oneway(data_2021[:,0],data_2021[:,1],data_2021[:,2],data_2021[:,3])
print(anova_result)

# 귀무가설 : mean 값에 차이가 없음
# p-value > 0.05, 귀무가설 수용! -> 평균값의 차이가 없다(2021년도)
# 결론 : 2021년도 4개 시도의 미세먼지 PM10 측정량 평균은 차이가 없다

F_onewayResult(statistic=0.5352888611262938, pvalue=0.6605335914872638)


In [20]:
# T-test
stats.ttest_ind(np_data[:,0], np_data[:,1])

Ttest_indResult(statistic=0.6169137946709962, pvalue=0.537827303526871)

In [21]:
# print(row_index, row_index.shape)
# print(np_data, np_data.shape)

# 필터 만들기
a = row_index.astype(np.float64)
b = (a%1.0)*100

filter_spring = (b>2) & (b<5)
# print(filter_spring)

filter_winter = (b>9) & (b<12)
# print(filter_winter)

spring_data = np_data[filter_spring].flatten()
winter_data = np_data[filter_winter].flatten()
print(spring_data, spring_data.shape)
print(winter_data, winter_data.shape)

[65 58 59 48 56 56 52 50 72 76 67 76 46 47 46 45 51 53 49 41 52 58 53 51
 55 66 54 56 52 49 48 46 55 62 53 58 60 50 50 57 58 53 45 51 63 71 59 81
 71 56 68 62 45 48 40 48 45 56 49 42 64 52 59 53 71 60 64 61 56 53 49 56
 60 52 54 47 56 54 58 59 63 58 63 50 52 44 52 43 52 63 60 56 42 44 47 38
 69 51 68 56 41 40 40 43 52 47 46 47 45 33 40 39 44 36 41 44 35 32 32 38
 67 60 63 76 42 37 41 37 61 36 56 44] (132,)
[42 43 42 40 38 41 40 36 46 41 43 36 33 42 33 37 42 47 44 44 41 36 38 26
 29 38 30 39 42 43 39 50 55 47 46 45 33 39 29 38 45 47 39 40 44 42 43 49
 44 43 50 51 33 33 37 41 48 41 49 42 38 37 36 42 52 49 51 43 48 44 47 38
 29 32 36 30 42 50 51 45 50 46 49 37 28 32 31 31 52 47 63 42 43 40 51 33
 31 27 29 34 40 35 43 44 42 35 40 33 33 30 33 34 42 31 37 31 42 33 38 28
 27 23 27 24 45 34 38 38 39 31 40 30] (132,)


In [22]:
# 평균값 구하기 & T-test
print(np.mean(spring_data), np.mean(winter_data))
print(stats.ttest_ind(spring_data, winter_data))

52.67424242424242 39.46969696969697
Ttest_indResult(statistic=12.232958045607598, pvalue=1.6003353734189516e-27)


In [23]:
# 2011 ~ 2021년까지 각 도시, 연도별 미세먼지 농도가 가장 높았던 월
# ex> 2021년도 부산 __월 미세먼지 농도가 가장 높았다.

for yr in range(2011, 2022) :
  start = 12*(yr-2011)
  stop = start+12
  yr_data = np_data[start:stop]
  # print(start,stop)
  # print(yr_data)
  # 연도별, 시도별 가장 농도가 높았던 월(인덱스를 구해 +1을 하여 월로 표현)
  print(np.argmax(yr_data, axis=0)+1)

[2 5 5 5]
[1 5 5 5]
[1 3 1 5]
[5 5 5 5]
[2 2 2 2]
[4 4 4 4]
[5 5 5 4]
[ 2  4 11  4]
[3 3 3 3]
[3 4 1 4]
[3 3 3 3]
